# 1. Language Models - Introduction, Markov Chains and N-Grams

> “You shall know the nature of a word by the company it keeps.” – John
> Rupert Firth

Text Generation(NLG) is ubiquitous in many NLP tasks, from summarization, to dialogue and machine translation. And a Language Model plays it part in each one of them. The goal of language modelling is to estimate the probability distribution of various linguistic units, e.g., words, sentences etc.([Source](https://medium.com/syncedreview/language-model-a-survey-of-the-state-of-the-art-technology-64d1a2e5a466)) In essence, we want our language model to learn the "grammar" and "style" of the text on which it was trained on.

**Key concepts to know beforehand:**

- [Probability](https://tinyheero.github.io/2016/03/20/basic-prob.html)
	- Joint Probability
	- Conditional Probability
 - [N-Grams](https://towardsdatascience.com/understanding-word-n-grams-and-n-gram-probability-in-natural-language-processing-9d9eef0fa058)
 - [Markov Chains](https://setosa.io/ev/markov-chains/)
 - [Entropy, Cross Entropy](https://deep-and-shallow.com/2020/01/09/deep-learning-and-information-theory/)
- Basic familiarity with NLP

## Imports

In [16]:
import nltk
import random
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import pprint
from IPython.display import display, Markdown, Latex

## Toy Data  

To illustrate and learn what Language Models are, we have taken the example - a tongue twister

In [5]:
data = """To sit in solemn silence in a dull, dark dock,
In a pestilential prison, with a life-long lock,
Awaiting the sensation of a short, sharp shock,
From a cheap and chippy chopper on a big black block,
To sit in solemn silence in a dull, dark dock,
In a pestilential prison, with a life-long lock,
Awaiting the sensation of a short, sharp shock,
From a cheap and chippy chopper on a big black block,
A dull, dark dock, a life-long lock,
A short, sharp shock, a big black block,
To sit in solemn silence in a pestilential prison,
And awaiting the sensation
From a cheap and chippy chopper on a big black block!"""

## Language Model   

As discussed before, a Language Model learns the "grammar" and "style" of the text on which it was trained on. But how do we translate that to a mathematical statement? We know that there is no hard and fast rules in language. An idea can be expressed in a lot of different ways. Therefore stochasticity is built into language. So, it follows that the mathematical model we aim for language is also probabilistic.

An LM (or what we casually refer to as an LM) is typically comprised of a few components as shown below.
![LM Overview](images/lm_overview.png)


1. The core Language Model which learns the "style and grammar" of the language.
2. A mechanism to generate robust probability distribution over words - eg. Smoothing or Backoff in Statistical Language Models
3. A mechanism to decode the probability distribution and generate text - eg. Greedy Decoding, Beam Search, top-k sampling, etc.
4. A downstream task - eg. Machine Translation, Text Summarization, etc.

There are two types of Language Models:
1. Count-based Language Models or Statistical Language Models
2. Continuous-space Language Models or Neural Language Models

Let's start with Count-based Language models as it is a very good starting point to understand Language Models. And once you have understood the rest of the components, it is very easy to switch out the LM block with more recent and popular Neural LMs like BERT, GPT, etc.

One of the ways of looking at a Language model is that it is a probabilistic model which knows the probability of a given word, provided the context. How can we formalize that? If the context can be defined as the preceeding words, then the Language Model becomes:   
$ P(word | all\;previous\;words\;in\;context)$ or more formally,  
$ P(w_n|w_1, w_2, w_3,...w_{n-1})$  

Simple enough right? We can get away with simple counting, because after all this is Probability. But we have a couple of problems. 
1. What do we do with the first word in a sentence? It doesn't start with any other. So even with a context window on 2, we cannot estimate probabilities for that.
2. What do we do when we are looking at a word somewhether in the middle of a large corpus? Our context will be half the corpus. This will get out of hand very very fast. It's just not computationally tractable to look at the entire history as the context.

For problem 1, there is a very easy and straightforward solution. We add **<start>** and **<end>** tokens to the sentences. This let's us model the probability of the first word, given <start> token and also provide the model a token with which it can stop a sentence as well.

For problem 2, we have the Markov Assumption

### Tokenization and Special Tokens 

To make things simple, we are just doing a split by word tokenization. But we also add **start** and **end** tokens

In [12]:
tokens = []
for sentence in data.split(","):
    tokens = tokens + [word.lower().strip() for word in sentence.lower().split()]

tokens = ['<start>'] + tokens + ['end']

tokens[:10]

['<start>', 'to', 'sit', 'in', 'solemn', 'silence', 'in', 'a', 'dull', 'dark']

### Markov Assumption   
So, we take the help of [Markov Property](https://en.wikipedia.org/wiki/Markov_property) to reduce the complexity of our Language Model. [Markov Property](https://en.wikipedia.org/wiki/Markov_property) states that the probability of any state depends only on the previous state and not on the entire sequence of events which preceded it. This helps us in dealing with the infinite context problem in the Language Model. Applying the Markov assumption to our Language Model, we can consider that the probability of the next word is only dependent on a fixed window(words in that window is the previous state) and not the entire history. This makes the language model simple enough to be feasible.

$P(w_i|w_1,w_2,w_3...w_{i-1}) \approx P(w_i|w_i-k ... w_{i-1})$

If we consider the last two words as the state, the language model becomes:

$P(w|w_{n-1}, w_{n-2})$

## Language Model using Markov Chains

### Markov Chains  

A Markov Chain is a stochastic process which statisfies Markov Property, i.e. a process where the past and future are independent of the present state(in our case, the words in the window of context). The most intuitive way of understanding the Language model is by using the Markov Chains directly. 

We can create a Markov Chain by considering multiple states and the transition probabilities from one state to the other. i.e. If my current state is $(w_{i-1}, w_{i-2}, w_{i-3})$, what is the probability that I move to $(w_{i}, w_{i-1}, w_{i-2})$. This can be seen as the probability that $w_i$ occurs after $(w_{i-1}, w_{i-2}, w_{i-3})$, and that can easily be estimated from the corpus.

So, first we need to make context-target pairs out of our tokens. Let's define window as 3 i.e. we are looking back three tokens as context

In [11]:
context_target_pairs = []
n=2
for i in range(len(tokens)-n):
        context, word = tokens[i:i+n], tokens[i+n]
        context_target_pairs.append((" ".join(context),word))
context_target_pairs[:10]

[('<start> to', 'sit'),
 ('to sit', 'in'),
 ('sit in', 'solemn'),
 ('in solemn', 'silence'),
 ('solemn silence', 'in'),
 ('silence in', 'a'),
 ('in a', 'dull'),
 ('a dull', 'dark'),
 ('dull dark', 'dock'),
 ('dark dock', 'in')]

Now that we have the Context-Target pairs, we can calculate the probability of the coming word, given a context by simple counting. 

Now, let's see an example. In our short toy corpus, what is the probability that **'dull'** comes after **'in a'**?  
The simplest way is to count the number of times **dull** occured after **'in a'** and then divide it by the number of times any word occured after **'in a'**. 

We initialize a *defaultdict* with a Counter inside to count the number of times a target comes after the context.

In [13]:
def count_markov_chain(context_target_pairs, order=4):
    mc = defaultdict(Counter)
    for context, target in context_target_pairs:
        mc[context][target]+=1
    return mc

In [14]:
mc = count_markov_chain(context_target_pairs, order=n)

Let's see all the words which occured after **'in a'**

In [15]:
mc["in a"]

Counter({'dull': 2, 'pestilential': 3})

$P(dull| in\;a) = \frac{C_{dull| in\;a}}{C_{in\;a}}$   
$P(dull | in\;a) = \frac {2}{5} = 0.4$

#### Side Note: ConditionalFreqDist   

The above can also be easily done with ConditionalFreqDist in nltk., which gives us richer objects instead of vanilla Counters and defaultdicts

In [14]:
mc = nltk.ConditionalFreqDist(context_target_pairs)
mc["in a"]

FreqDist({'pestilential': 3, 'dull': 2})

## Language Model from N-Grams

There is an alternate way of calculating the probabilities for the Language Model, although not as intuitive as the previous one.

Let's start by taking a hard look at the formula from previous section
$P(dull| in\;a) = \frac{C_{dull| in\;a}}{C_{in\;a}}$      

What is this $C_{dull| in\;a}$? Isn't it just the count of the trigram - **in a dull**?     

And $C_{in\;a}$ is just the count of the bigram - **in a**.

If we generalize this to n-grams, 

$P(w_i|w_{i-n+1}^{i-1}) = \frac{C(w_{i-n+1}^{i})}{C(w_{i-n+1}^{i-1})} = \frac{C(n\;gram)}{C(n-1\;gram)}$   

It's good to know both of these are the same because in the literature around, you will find both and may be confused as to which one is correct. Fear, not both are. NLTK uses the previous methodology in it's LMs(which is what we are going to use soon.)


NLTK has a lot of ready-to-use functions for the manipulation of corpus, sentences, words etc.

In [19]:
trigrams = list(nltk.ngrams(tokens, n=3))
bigrams = list(nltk.ngrams(tokens, n=2))
trigram_freq = nltk.FreqDist([" ".join([w1,w2,w3]) for w1, w2, w3 in trigrams])
bigram_freq = nltk.FreqDist([" ".join([w1,w2]) for w1, w2 in bigrams])

bigram_count = bigram_freq['in a']
trigram_count = trigram_freq['in a dull']
display (Markdown(f"Count of Trigram **in a dull**: {trigram_count}. Count of bigram **in a.**: {bigram_count}"))
display (Latex(r"$P(dull| in\;a) = \frac{" + str(trigram_count)+"}{" +str(bigram_count)+"} = "+str(trigram_count/bigram_count)+"$"))

Count of Trigram **in a dull**: 2. Count of bigram **in a.**: 5

<IPython.core.display.Latex object>

This is exactly the probability we estimated through the nmarkov chain method.

## Evaluation  

The evaluation of NLG models is a long debated and actively researched topic. [Hashimoto et al., 2019](https://nlp.stanford.edu/pubs/hashimoto2019unifying.pdf) proposes that _"A good evaluation metric should not only capture the **quality of generation**, but also the **diversity of generation**, which is especially crucial for creative, open-ended tasks like dialogue or story generation."_ 

Human Evaluation, which is considered as the gold standard in NLG, only captures quality of the generated text while ignores diversity. for eg. If a Language Model regurgitates a sentence from the training corpus, it will pass the human evaluation with flying colors. But does the model have the generalization it shoyuld have?

Statistical Evaluation metrics like Perplexity captures diversity and ensures the model assigns reasonable probability to unseen sentences.

There are automated ways of imitating Human Evaluation - [BLEU, METEOR, and ROUGE](https://medium.com/explorations-in-language-and-learning/metrics-for-nlg-evaluation-c89b6a781054) are metrics proposed by [Papineni et al., 2001](https://www.aclweb.org/anthology/P02-1040.pdf), [Banerjee et al., 2005](https://www.cs.cmu.edu/~alavie/METEOR/pdf/Banerjee-Lavie-2005-METEOR.pdf), and [Lin et al., 2004](https://www.aclweb.org/anthology/W04-1013.pdf), respectively. BLEU and METEOR are the de-facto standard in Machine Translation and ROUGE is often used in Text Summarization. Criticism of these metrics are also very prominent in the NLProc community. Multiple studies have shown that Human Evaluation and these automatic metrics do not enough correlation. Rachel Tatman has written an excellent [blog](https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213) on the subject warning us of using BLUE or any other automatic metric without really understanding it.

## Perplexity

Perplexity is a close cousin of [Entropy](https://deep-and-shallow.com/2020/01/09/deep-learning-and-information-theory/). Wikipedia defines Perplexity as a measurement of how well a probability distribution or probability model predicts a sample. It can also seen as how well the language model anticipates unseen data.

Mathematically, 
$Perplexity\;for\;a\;Probability\;Distribution = b^{H(p)}$, where $H(p)$ is the Entropy of the distribution and $b$ is the base of the log used, most commonly 2 or e.
$Entropy = \sum_{i=1}^{n} p(x_i)log_b(p(x_i))$

So, we already know that Entropy is the expected number of bits needed to encode the information contained in a random variable. Perplexity is just the exponentiation of the same, which means it has the same information, but just expressed differently.

Let's take a look at how we can apply this to Language Modelling(If you have not read my previous blogpost on Entropy, I urge you to. It will give you the intuition required for  further discussions).

In Language Modelling, we are building a model with a distribution $q$ over the words which tries to mimic the real world distribution of $p$ over words as close as possible. In practice we do not know $p$, but only sampled $p'$ from the language. So, we use the definition of Perplexity with this sampled $p'$, $q$ and the Cross Entropy between these two.

If we sample N sentences from the Language,
$PP(p',q) = b^{-\sum_{i=1}^{N} p'(x_i)log_b(q(x_i))}$, where $p(x_i)$ is the sampled probability of sentence i and $q(x_i)$ is the probability assigned to that sentence by our language model.

For a reasonably large dataset we assume the probability $p(x_i) = \frac{1}{N}$. 
(**Note:** This is more to give you an intuition about the assumptions made. For exact mathematical proof, check out The Shannon-McMillan-Breiman Theorem which states that when N is sufficiently large, the $p(x_i)$ is automatically inferred by the distribution of the sample we are working with.)

And we already know $q(x) = P(w_1, w_2, ....w_k)$, for a sentence with k words.
Using some bit of Probability magic, it becomes:

$q(x) = \prod_{j=1}^k P(w_j|w_1, w_2,...w_{j-1}$, which is nothing but the product of all the word probabilities estimated by our language model.

Putting these back into the exponent,

$-\frac{1}{N}\sum_{i=1}^{N} log_b(q(x_i))$   

Taking the $\frac{1}{N}$ inside the log term, it becomes,   

$\sum_{i=1}^{N} log_b(q(x_i^{-\frac{1}{N}}))$.

Now, using the log-exponent rule ($b^{log_b{x}} = x$), we rewrite the perplexity equation as,

$PP(p',q) = \sum_{i=1}^{N} log_b(q(x_i^{-\frac{1}{N}}))$

Simplifying it and removing Log,

$PP(p',q) = (\prod_{i=1}^{N} q(x_i))^{-\frac{1}{N}}$    

And this is the perplexity which is used in NLP as a measure of uncertainity of the language model. Lower the perplexity, better the model.

Intuitively, if model is a good language model, the probability it assigns to a test set would be quite high; because the model is not surprised/perplexed by the test set data. And when the probability is high, Perplexity is low.

## Perplexity and Meena

[Adiwardhana et al., 2020](https://arxiv.org/pdf/2001.09977.pdf) introduced a large open-domain Conversational agent, Meena, introduced a new metric to evaluate the responses - Sensibleness and Specificity Average(SSA). This was a metric used to measure both the sensibleness and specificity of a human like chatbot and used a set of crowd workers to evaluate the output generated by the model. But for quick research iterations, they had chosen Perplexity as the measure. And what they found out at the end was that Perplexity had a large correlation(~0.95) with the SSA metric, which makes a great case to use Perplexity as the metric to optimize, epecially for open domain Convesational Chatbots 

## Sample Calculation
Let's take an example from our corpus and calculate Perplexity. Here I have constructed a new sentence, from the markov chain as test data.

In [357]:
test_data = "to sit in solemn silence in a pestilential prison and awaiting the sensation from a cheap and chippy chopper"
fake_data = "Jack and Jill went to the prison"

In [361]:
#Not adding <end> token for illustration
test_tokens = ["<start>"]+[word.strip().lower() for word in test_data.split()]

First, let's calculate the Probability of the entire sentence.

In [366]:
#Creating trigrams as the window. First two is the context and last one is the target
trigrams = nltk.trigrams(test_tokens)
#Initialize the probability as 1 and using the ConditionalFreqDist from nltk as the model
prob = 1
for trigram in trigrams:
    _p = mc[" ".join(trigram[:2])][trigram[-1]]/mc[" ".join(trigram[:2])].N()
    print(trigram, _p)
    prob = prob * _p
print("Probability of the sentence: {}".format(prob))

('<start>', 'to', 'sit') 1.0
('to', 'sit', 'in') 1.0
('sit', 'in', 'solemn') 1.0
('in', 'solemn', 'silence') 1.0
('solemn', 'silence', 'in') 1.0
('silence', 'in', 'a') 1.0
('in', 'a', 'pestilential') 0.6
('a', 'pestilential', 'prison') 1.0
('pestilential', 'prison', 'and') 0.3333333333333333
('prison', 'and', 'awaiting') 1.0
('and', 'awaiting', 'the') 1.0
('awaiting', 'the', 'sensation') 1.0
('the', 'sensation', 'from') 0.3333333333333333
('sensation', 'from', 'a') 1.0
('from', 'a', 'cheap') 1.0
('a', 'cheap', 'and') 1.0
('cheap', 'and', 'chippy') 1.0
('and', 'chippy', 'chopper') 1.0
Probability of the sentence: 0.06666666666666665


Now, the perplexity is just the Nth root of the inverse of probability. In practice, we would add the log probabilities to avoid numeric underflow.

In [370]:
perplexity = pow(1/prob, -(1/len(tokens)))
print(f"Perplexity: {perplexity}")

Perplexity: 0.9767268341369304


### Generalization and Out of Vocabulary Words

The simplistic approach to Language Modelling we saw till now has some problems when it comes to generalizing out of sample data. There are two ways this problem can manifest itself
1. A totally new word that the model hasn't seen shows up.
2. A totally new context that the model hasn't seen shows up.

Let's face it. No matter how huge the corpus is going to be, we are not going to cover every single possible combinations word tokens in the language. Neither all the words in the language. And this means that sooner or later, we are going to stumble across unseen data/OOV tokens. 

And let's see what happens when we lookup the probability of an out of vocabulary token.   

Let's say the OOV token is **exotic**, given the context **in a**.

In the naive way:

In [368]:
mc['in a']

FreqDist({'pestilential': 3, 'dull': 2})

$P(exotic| in\;a) = \frac{C_{exotic| in\;a}}{C_{in\;a}} = \frac{0}{5} = 0$   

And this zero probability may look harmless at fiorst glance, but let's think about what would happen to Perplexity if there is an OOv token. We multiple the probabilities of all the n-grams in the test corpus and even if we have a single OOV word, it would make the entire probability zero. And the inverse of the probability becomes infinity and by extension perplexity also infinity.


Let's see how to deal with OOV in a later point of time. In the next part, let's try and formalize our intuition using an implementation of the LanguageModel from NLTK